In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import math
import numpy as np
from functools import reduce

In [2]:
"""
n = dimension of input
m = dimension of output
"""

'\nn = dimension of input\nm = dimension of output\n'

In [3]:
def calc_logit(input, weight, bias=None):
    """
    Calculate logit = input.mm(weight.t()) + bias

    Args:
        input:  [1 x n]         Input vector
        weight:  [m x n]        Weight matrix
        bias:  [m]              Bias vector

    Returns:
        logit: [1 x n]          Logit = input.mm(weight.t()) + bias

    """

    logit = input.mm(weight.t())
    if bias is not None:
        logit += bias.unsqueeze(0).expand_as(logit)

    return logit

## ESGD

In [4]:
def esgd_arctan(grad_output, input, weight, bias, output):

    grad_output_scaled = grad_output / (1 + logit ** 2)  # [1 x m]
    grad_input = grad_output_scaled.mm(weight)  # [1 x n]
    grad_weight = grad_output_scaled.t().mm(input)  # [m x n]
    grad_bias = grad_output_scaled.sum(0).squeeze(0)  # [m]

    return grad_input, grad_weight, grad_bias

## Old ISGD implementation

In [5]:
def calc_backwards_variables(saved_tensors, logit, grad_output, lr, mu):
    """
    Calculate the variables required for back-propagation
    
    Args:
        saved_tensors:          Stores from forward-propagation the input, weight, bias, output
        logit: [1 x n]          Stores from forward-propagation the logit
        grad_output: [1 x m]    The gradient that has been back-propagated to this layer
        lr: [1]                 Learning rate
        mu: [1]                 Ridge-regularization constant

    Returns:
        input: [1 x n]          Input vector
        weight: [m x n]         Weight matrix
        bias: [m]               Bias vector
        output [1 x m]          Input to the next layer = logit put through the non-linear activation function
        logit: [1 x n]          Logit
        s: [1 x m]              Sign of back-propagated gradient
        z_norm: [1]             2-norm of (input, 1)
        d: [1 x m]              Weighted constant, proportional to the sqrt(abs(back-propagated gradient))
        c: [1 x m]              Logit contracted by ridge-regularization
    """

    # Unpack saved values
    input, weight, bias, output = saved_tensors

    # ISGD constants
    s = torch.sign(grad_output)
    z_norm = math.sqrt((torch.norm(input) ** 2 + 1.0))
    d = z_norm / math.sqrt(1.0 + lr * mu) * torch.sqrt(torch.abs(grad_output))
    c = logit / (1.0 + lr * mu)

    return input, weight, bias, output, logit, s, z_norm, d, c

In [6]:
def calc_weigh_bias_grad(weight, mu, lr, a, d, input, z_norm, bias):
    """
    Calculate the gradient of the weight matrix and bias vector

    Args:
        weight: [m x n]         Weight matrix
        mu: [1]                 Ridge-regularization constant
        lr: [1]                 Learning rate
        a: [1 x m]              Solution of ISGD update
        d: [1 x m]              Weighted constant, proportional to the sqrt(abs(back-propagated gradient))
        input: [1 x n]          Input vector
        z_norm: [1]             2-norm of (input, 1)
        bias: [m]               Bias vector

    Returns:
        grad_weight: [m x n]    Gradient of the weight matrix
        grad_bias: [m]          Gradient of the bias vector

    """

    grad_weight = weight * mu / (1.0 + lr * mu) - (a * d).t().mm(input) / z_norm ** 2
    grad_bias = bias * mu / (1.0 + lr * mu) - (a * d).squeeze() / z_norm ** 2
    return grad_weight, grad_bias

In [7]:
def real_root_closest_to_zero(coeff):
    roots = np.roots(coeff)
    real_roots = [root.real for root in roots if root.imag == 0]
    root_closest_to_zero = reduce((lambda x, y: x if (abs(x) < abs(y)) else y), real_roots)
    return root_closest_to_zero

In [9]:
def isgd_arctan(grad_output, input, weight, bias, output, logit):
    
    lr, mu = 0.00000001, 0.0
    input, weight, bias, output, logit, s, z_norm, d, c = calc_backwards_variables([input, weight, bias, output], logit,
                                                                                       grad_output, lr, mu)

    # Calculate a
    coeff = np.array([((lr * d) ** 2).numpy()[0],
                      (2 * lr * d * c).numpy()[0],
                      (c ** 2 + 1).numpy()[0],
                      (s * d).numpy()[0]])

    root_closest_to_zero = np.apply_along_axis(real_root_closest_to_zero, 0, coeff)
    a = torch.from_numpy(root_closest_to_zero).unsqueeze(1).t().type(torch.FloatTensor)

    # Calculate grad_weight, grad_bias and return all gradients
    grad_weight, grad_bias = calc_weigh_bias_grad(weight, mu, lr, a, d, input, z_norm, bias)

    # Calculate input gradient
    grad_output_scaled = grad_output / (1 + logit ** 2)  # [1 x m]
    grad_input = grad_output_scaled.mm(weight)  # [1 x n]

    return grad_input, grad_weight, grad_bias

In [68]:
def isgd_arctan2(grad_output, input, weight, bias, output, logit):
    
    lr, mu = 0.0, 0.0
    input, weight, bias, output, logit, s, z_norm, d, c = calc_backwards_variables([input, weight, bias, output], logit,
                                                                                       grad_output, lr, mu)

    # Calculate a
    d_d = d.double()
    s_m_d = (s * d).double()
    c_d = c.double()

    a = d_d * 0  # [b x m]
    a_diff = 1  # Norm difference between previous and current a values
    iter_count = 0  # Count of number of a iterations
    while a_diff > 1e-15:
        a_new = - s_m_d / (1.0 + (lr * d_d * a + c_d) ** 2)  # [b x m]
        a_diff = torch.norm(a - a_new)
        a = a_new  # [b x m]
        iter_count += 1
        if iter_count >= 50:
            assert (iter_count < 50), 'Arctan update has failed to converge'

    # Make a float so that can be operated on with other tensors
    a = a.float()

    # Calculate grad_weight, grad_bias and return all gradients
    grad_weight, grad_bias = calc_weigh_bias_grad(weight, mu, lr, a, d, input, z_norm, bias)

    # Calculate input gradient
    grad_output_scaled = grad_output / (1 + logit ** 2)  # [b x m]
    grad_input = grad_output_scaled.mm(weight)  # [b x n]

    return grad_input, grad_weight, grad_bias

## New ISGD implementation

In [215]:
def isgd_new_arctan(input, weight, bias, output, logit, grad_output):
    
    # Hyperparameters
    lr = 0.0
    mu = 0.0

    # ISGD constants
    s = torch.sign(grad_output)  # [b x m]
    z_norm_squared = torch.norm(input, p=2, dim=1) ** 2 + 1.0  # [b]
    c = logit / (1.0 + lr * mu)  # [b x m]

    # Calculate u
    # Calculate conditions for u
    
    # Make everything doubles to prevent rounding errors
    b = grad_output.double() / (1.0 + lr * mu)
    c_d = c.double()
    z_d = z_norm_squared.double()

    u = c_d * 0  # [b x m]
    u_diff = 1  # Norm difference between previous and current a values
    iter_count = 0  # Count of number of a iterations
    while u_diff > 1e-10:
        u_new = b / (1.0 + (c_d - lr * torch.mul(z_d, u.t()).t()) ** 2) # [b x m]
        u_diff = torch.norm(u - u_new)
        u = u_new  # [b x m]
        iter_count += 1
        if iter_count >= 50:
            assert (iter_count < 50), 'Arctan update has failed to converge'

    # Make a float so that can be operated on with other tensors
    u = u.float()

    # Calculate input gradient
    grad_output_scaled = grad_output / (1 + logit ** 2)  # [b x m]
    grad_input = grad_output_scaled.mm(weight)  # [b x n]

    # Calculate grad_weight, grad_bias
    grad_weight = weight * mu / (1.0 + lr * mu) + u.t().mm(input)  # [m x n]
    grad_bias = bias * mu / (1.0 + lr * mu) + u.t().sum(1)  # [m]
    
    return grad_input, grad_weight, grad_bias

## Test the differences between ESGD, ISGD old and ISGD_new

In [242]:
# Random data
grad_output = torch.randn(1, 3)     # [1 x m]
input = torch.randn(1, 2)           # [1 x n]
weight = torch.randn(3, 2)          # [m x n]
bias = torch.randn(3,)              # [m]

# Forward propagation
# Calculate logit [1 x m], where logit = input.mm(weight.t()) + bias
logit = calc_logit(input, weight, bias)

# Non-linear activation function
output = torch.atan(logit)  # [1 x m]

# Calculate gradients
esgd_grads = esgd_arctan(grad_output, input, weight, bias, output)
isgd_grads = isgd_arctan2(grad_output, input, weight, bias, output, logit)
isgd_new_grads = isgd_new_arctan(input, weight, bias, output, logit, grad_output)

# Print difference
# print('Difference between ESGD and ISGD old')
# print([(x-y) for x,y in zip(isgd_grads, esgd_grads)])

print('\nDifference between ESGD and ISGD new')
print([(x-y) for x,y in zip(isgd_new_grads, esgd_grads)])


Difference between ESGD and ISGD new
[
 0  0
[torch.FloatTensor of size 1x2]
, 
 0  0
 0  0
 0  0
[torch.FloatTensor of size 3x2]
, 
 0
 0
 0
[torch.FloatTensor of size 3]
]
